#### Pseudo Code

- [X] Parse "Curve Header Data" tab from curve PSD
- [X] Create dictionary/dataframe?
- [X] Create Root element and namespace ('SKBData')
- [X] Add Child Element and attributes('CurveFamily')
- [ ] Convert and insert header_dict as grandchildren of CurveFamily Element
- [ ] Create (w/ modified values) and Insert 'pumpCurveCollection' element for each tab in NBS Curve PSD
- [ ] Create and Insert 'Impeller' Element as children to 'pumpCurveCollection' element.

#### File Setup

In [20]:
from lxml import etree as ET
import os
import pandas as pd
import re

In [21]:
# fileDir = r"C:\Users\104092\OneDrive - Grundfos\Documents\20-29 Areas\22 grundfos-express-tools\curve PSD to XML"
# filename = "Curve_Alpha_1Aug2022.xml"
# filepath = os.path.join(fileDir, filename)

psd_path = r"C:\Users\104092\OneDrive - Grundfos\Documents\10-19 Projects\12 NBS Curve PSD Separation\12.02 Output Files"
psd_file = r"GXS Curve_Conexus_V2 - std models.xlsx"
psd_filepath = os.path.join(psd_path, psd_file)

#### Create Root Element (and namespace)

In [22]:
def add_namespace(elem_tag, xsi_namespace):
    XHTML_NAMESPACE = xsi_namespace
    XHTML = "{%s}" % XHTML_NAMESPACE
    NSMAP = {'xsi' : XHTML_NAMESPACE} # the default namespace (no prefix)

    return ET.Element(elem_tag, nsmap=NSMAP) # lxml only!

In [23]:
root_ns = "http://www.w3.org/2001/XMLSchema-instance"
root = add_namespace('SKBData', root_ns)

#### Create CurveFamily Element as subelem to root

In [24]:
curveFamily_elem = ET.SubElement(root, "CurveFamily", selectorVersion="8.0.0", skbVersion="22.2.0.220418.623")

curve_family_name = "NBS_Fixed_Trim"

header_dict = {
        'name': curve_family_name}
        # 'svDataType':'speed',
        # 'interpDataType':'speed',
        # 'compressorConditionsInputTypeSkb':'speedofSound',
        # 'interpDataType':'speed',
        # 'compressorConditionsInputTypeSkb':'speedOfSound',
        # 'flowTypeSkb':'volumetricFlow',
        # 'headTypeSkb':'head',
        # 'headMarginForFixedDiameter':'value',
        # 'submergenceMethod':'fixedValue',
        # 'errorFitMax':'1.5',
        # 'pumpType':'0',
        # 'interpQty':'3',
        # 'efficiencyPowerDataType':'pump'}

# Add header data as sub-elements to CurveFamily Tag       
for key, value in header_dict.items():
    elem = ET.SubElement(curveFamily_elem, key)
    elem.text = value

#### Inserting UnitofMeasureSettings as subelement to CurveFamily

In [25]:
# unitOfMeasureSettings
with open('unitofmeasuresettings-boilerplate.txt','r') as file:
	unit_of_measure_settings = file.read()
	
uom_element = ET.fromstring(unit_of_measure_settings)
curveFamily_elem.append(uom_element)

#### Currently In Progress. Add pumpCurveCollection for each curve tab

In [26]:
def create_impeller_dict(curve_number):

    # Extract diameter (in mm) from curve name
    res = re.search("-(\d+)_Std", curve_number)
    curve_trim_size_mm = int(res.group(1))
    curve_trim_size_in = curve_trim_size_mm/25.4

    impeller_dict = {'diameter': curve_trim_size_in}

    return impeller_dict

In [27]:
def create_pump_curve_dict(row) -> dict:
    """Returns dictionary of updated attributes to be converted to elements """
    pumpCurve_dict = {
        'curveNumber': row['Curve number'],
        'speedRef': row['Speed, data'],
        'polesRef': row['Poles'],
        'hzRef': row['Hz'],
        'mcsfMinRef': row['MCSF @ min impeller diameter'],
        'mcsfMaxRef': row['MCSF @ max impeller diameter'],
        'eyeCount': row['Number of impeller eyes'],
        'speedCurveNominal': row['Speed, nominal'],
        'speedCurveMin': row['Speed, Min'],
        'speedCurveMax': row['Speed, max'],
        'diaImpInc': row['Diameter increment'],
        'speedVariableCurveMin': row['Variable speed min limit'],
        'optionalCurveType': 'Power',
        'flowStartHeadEnabled': 'false',
        'flowStartEtaEnabled': 'false',
        'flowStartPowerEnabled': 'false'
    }   

    return pumpCurve_dict

In [28]:
def add_elem_from_dict(parent_elem, elem_dict):
    """Takes elements inside elem_dict and adds as elements to parent_elem"""
    for key, value in elem_dict.items():
        elem = ET.SubElement(parent_elem, key)
        elem.text = str(value)

In [29]:
def metric_to_us(input_value, parameter_type:str):
    if parameter_type == 'flow':
        return input_value * 4.40286764029913
    elif parameter_type == 'distance':
        return input_value * 3.28083989501312
    elif parameter_type == 'power':
        return input_value * 1.3410218586563
    else:
        print("non valid parameter type entered")

In [30]:
def add_curve_data_points(parent_elem, curve_number, curve_type):
    
    curve_data_df = pd.read_excel(psd_filepath,sheet_name=curve_number, header=7, skiprows=[8], usecols="D,E,L,S", nrows=50)
    curve_data_df = curve_data_df.dropna()
    
    for index, row in curve_data_df.iterrows():
        datapoint_elem = ET.SubElement(parent_elem, "DataPoint", disabled="false")
        
        if curve_type == 'Power':
            datapoint_dict = {
                # 'x': metric_to_us(row['Flow'], "flow"),
                # 'y': metric_to_us(row[curve_type], 'power'),
                'x': row['Flow'],
                'y': row[curve_type],
                'isOnCurve':'false',
                'division':'false',
                'slopeEnabled':'false'
            }

        elif (curve_type == 'Head') or (curve_type == 'NPSH'):
            datapoint_dict = {
                # 'x': metric_to_us(row['Flow'], "flow"),
                # 'y': metric_to_us(row[curve_type], 'distance'),
                'x': row['Flow'],
                'y': row[curve_type],
                'isOnCurve':'false',
                'division':'false',
                'slopeEnabled':'false'
            }

        else:
            print(f'curve_type not allowed: {curve_type}')
            
        add_elem_from_dict(datapoint_elem, datapoint_dict)

In [31]:
def add_curve(parent_elem, curve_type:str, curve_number):
    curve_elem = ET.SubElement(parent_elem, 'Curve', type=curve_type)

    # Add Curve Data Points to Curve Element
    add_curve_data_points(curve_elem, curve_number, curve_type)

In [32]:
curve_header_data = pd.read_excel(psd_filepath,sheet_name="Curve Header Data", header=8, skiprows=[9])

# Need to set appropriate values before adding to XML tree. Will need to iterate through curve_header_data to update values.
for index, row in curve_header_data.iterrows():
    
    # <pumpCurveCollection xsi:type="CentrifugalPumpCurveCollection"> This is the parent of each pump curve"
    qname = ET.QName(root_ns,"type")
    pumpCurveCollection_elem = ET.SubElement(curveFamily_elem, 'pumpCurveCollection', {qname: "CentrifugalPumpCurveCollection"})

    # Creates pump curve elements
    curve_dict = create_pump_curve_dict(row)
    add_elem_from_dict(pumpCurveCollection_elem, curve_dict)

    # Add Impeller Elements
    impeller_elem = ET.SubElement(pumpCurveCollection_elem, 'Impeller')
    impeller_dict = create_impeller_dict(row['Curve number'])
    add_elem_from_dict(impeller_elem, impeller_dict)

    # Add Curve Element
    add_curve(impeller_elem, "Head", row['Curve number'])
    add_curve(impeller_elem, "Power", row['Curve number'])
    add_curve(impeller_elem, "NPSH", row['Curve number'])    

In [ ]:
# print(ET.tostring(root, pretty_print=True).decode("utf-8"))

In [ ]:
# text_file = open("nbs_fixed_trim_all_curves.xml", "wt")
et = ET.ElementTree(root)
et.write('nbs_fixed_trim_all_curves.xml', pretty_print=True)
# n = text_file.write(root_as_str)
# text_file.close()